In [290]:
# Imports necesarios
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import xgboost as xgb

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.metrics import log_loss
from google.colab import files
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

In [291]:
pip install catboost

In [292]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lectura y Filtrado

In [293]:
short_df = pd.read_csv( "/content/drive/MyDrive/Orga Datos/TP2/DataFrame_train_encoded_2.csv" )
short_df.drop(columns='Unnamed: 0', inplace=True)
short_df

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
16767,0.0,0.0,1.0,0.0,0.0,11.702880,0.0,1.0,1.0,1.0
16768,0.0,0.0,1.0,0.0,0.0,11.597520,0.0,1.0,1.0,1.0
16769,0.0,0.0,1.0,0.0,0.0,11.702880,0.0,1.0,1.0,1.0
16770,0.0,1.0,0.0,0.0,0.0,14.668562,4.0,1.0,1.0,0.0


In [294]:
test = pd.read_csv( "/content/drive/MyDrive/Orga Datos/TP2/DataFrame_test_encoded_2.csv" )
opportunity_id = pd.DataFrame()
opportunity_id['Opportunity_ID'] = test['Opportunity_ID'] 
test.drop(columns=['Opportunity_ID', 'Unnamed: 0'], inplace=True)
test

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0.0,0.0,1.0,0.0,0.0,11.905436,0.0,1.0,1.0
1,0.0,0.0,1.0,0.0,0.0,11.787392,0.0,1.0,1.0
2,0.0,1.0,0.0,0.0,0.0,9.954062,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.0,12.338534,1.0,1.0,1.0
4,0.0,1.0,0.0,0.0,0.0,8.657390,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1562,0.0,0.0,1.0,0.0,0.0,11.733987,0.0,1.0,1.0
1563,0.0,1.0,0.0,0.0,0.0,10.715637,0.0,1.0,1.0
1564,0.0,1.0,0.0,0.0,0.0,11.514145,0.0,1.0,1.0
1565,0.0,1.0,0.0,0.0,0.0,11.872137,0.0,1.0,1.0


In [295]:
opportunity_id

,Opportunity_ID
0,10689
1,10690
2,10691
3,10692
4,10693
...,...
1562,12364
1563,12365
1564,12366
1565,12367


# Split

In [296]:
X = short_df.drop( 'Decision', axis=1 )
y = short_df['Decision']
X

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
16767,0.0,0.0,1.0,0.0,0.0,11.702880,0.0,1.0,1.0
16768,0.0,0.0,1.0,0.0,0.0,11.597520,0.0,1.0,1.0
16769,0.0,0.0,1.0,0.0,0.0,11.702880,0.0,1.0,1.0
16770,0.0,1.0,0.0,0.0,0.0,14.668562,4.0,1.0,1.0


In [297]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Creando columna "Tendence"

In [298]:
xg_classificator1 = XGBClassifier(colsample_bytree = 0.8, gamma = 1, learning_rate = 0.01, max_depth = 20, min_child_weight = 1, n_estimators = 200, subsample = 0.8)

In [299]:
xg_classificator1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1,
              learning_rate=0.01, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [300]:
tendence = xg_classificator1.predict(X)
tendence

array([0., 1., 1., ..., 1., 0., 0.])

In [301]:
tendence_test = xg_classificator1.predict(test)
tendence_test

array([1., 0., 1., ..., 0., 1., 0.])

In [302]:
#X['Tendence'] = tendence
#test['Tendence'] = tendence_test

In [303]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Linear Regresion

In [304]:
# Creamos el objeto de Regresión Linear
regr = linear_model.LinearRegression(normalize=True)


In [305]:
# Entrenamos nuestro modelo
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [306]:
# Hacemos las predicciones que en definitiva una línea (en este caso, al ser 2D)
LinReg_pred = regr.predict(X_test)

In [307]:
loss = log_loss(y_test, LinReg_pred)
print(f"Log loss is {loss}")

Log loss is 0.5030620577596426


# RandomForest

In [308]:
rfg = RandomForestRegressor()

In [309]:
rfg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [310]:
rfg_pred = rfg.predict(X_test)

In [311]:
loss = log_loss(y_test, rfg_pred)
print(f"Log loss is {loss}")

Log loss is 1.3718580283150215


# XGBoost

In [312]:
xg_Regressor1 = xgb.XGBRegressor(colsample_bytree= 0.8, gamma= 1.5, learning_rate= 0.01, max_depth= 20, min_child_weight= 1, n_estimators= 200, subsample= 0.8)

In [313]:
xg_Regressor1.fit(X_train,y_train)

[19:54:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1.5,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=20, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [314]:
xgbr = xg_Regressor1.predict(X_test)

In [315]:
loss = log_loss(y_test, xgbr)
print(f"Log loss is {loss}")

Log loss is 0.45383970831359466


# CatBoost

In [316]:
cbr = CatBoostRegressor(num_boost_round = 1500, learning_rate = 0.007)

cbr.fit(X_train, y_train)

0:	learn: 0.4933784	total: 3.27ms	remaining: 4.91s
1:	learn: 0.4914852	total: 5.38ms	remaining: 4.03s
2:	learn: 0.4896131	total: 7.66ms	remaining: 3.82s
3:	learn: 0.4877590	total: 10.8ms	remaining: 4.02s
4:	learn: 0.4859597	total: 12.1ms	remaining: 3.63s
5:	learn: 0.4841373	total: 14.5ms	remaining: 3.61s
6:	learn: 0.4823278	total: 17.1ms	remaining: 3.65s
7:	learn: 0.4805832	total: 18.5ms	remaining: 3.45s
8:	learn: 0.4788567	total: 20ms	remaining: 3.31s
9:	learn: 0.4770967	total: 22.7ms	remaining: 3.39s
10:	learn: 0.4753564	total: 25.4ms	remaining: 3.44s
11:	learn: 0.4736508	total: 27.2ms	remaining: 3.37s
12:	learn: 0.4719465	total: 29.8ms	remaining: 3.41s
13:	learn: 0.4702670	total: 32.7ms	remaining: 3.47s
14:	learn: 0.4686030	total: 35.7ms	remaining: 3.54s
15:	learn: 0.4669701	total: 38.5ms	remaining: 3.57s
16:	learn: 0.4653432	total: 41.2ms	remaining: 3.6s
17:	learn: 0.4637713	total: 42.8ms	remaining: 3.52s
18:	learn: 0.4621672	total: 45.5ms	remaining: 3.55s
19:	learn: 0.4605967	tota

In [317]:
cbr_pred = cbr.predict(X_test)

In [318]:
loss = log_loss(y_test, cbr_pred)
print(f"Log loss is {loss}")

Log loss is 0.46654223752902274


# LightBM

In [319]:
lbm = LGBMRegressor(boosting_type = "goss", n_estimators = 1100, learning_rate = 0.008, random_state = 123)

In [320]:
lbm.fit(X_train, y_train)

LGBMRegressor(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.008, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [321]:
lbm_pred = lbm.predict(X_test)

In [322]:
loss = log_loss(y_test, cbr_pred)
print(f"Log loss is {loss}")

Log loss is 0.46654223752902274


# KNN

In [323]:
n_neighbors = 6

knn = KNeighborsRegressor(n_neighbors, metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
                    metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                    weights='uniform')

In [324]:
knn_pred = knn.predict(X_test)

In [325]:
loss = log_loss(y_test, knn_pred)
print(f"Log loss is {loss}")

Log loss is 2.1333955732409913


# Ensamblando

In [ ]:
# Modelos totales: ('rfg', rfg), ('lbm', lbm),('xgbr', xg_Regressor1), ('lr', regr), ('cbr', cbr), ('knn', knn)
ereg = VotingRegressor([ ('rfg', rfg), ('lbm', lbm),('xgbr', xg_Regressor1), ('lr', regr), ('cbr', cbr)])
ereg.fit(X_train, y_train)

In [327]:
ereg_pred = ereg.predict(X_test)

In [328]:
loss = log_loss(y_test, ereg_pred)
print(f"Log loss is {loss}")

Log loss is 0.4997365399817264


In [329]:
ensamble = ereg.predict(test)

In [330]:
resultado = pd.DataFrame()
resultado['Opportunity_ID'] = opportunity_id['Opportunity_ID']
resultado['target'] = ensamble
resultado.sort_values('Opportunity_ID',inplace = True)
resultado = resultado.drop_duplicates('Opportunity_ID',keep = 'last')
resultado.set_index('Opportunity_ID', inplace = True)
resultado

,target
Opportunity_ID,
10689,0.720239
10690,0.170479
10691,0.711438
10692,0.153029
10693,0.729275
...,...
12364,0.260865
12365,0.290943
12366,0.301808


In [331]:
csv3 = resultado.to_csv('ensamble1.csv')
#files.download('ensamble1.csv')